### Generate boolean mask for LMEs
Running this code takes a few hours, so preferable to do it via the `.py` file : `generate_mask_LMEs.py`.

In [5]:
import geopandas as gpd
from information import *
import xarray as xr
from shapely.geometry import Polygon,Point
import numpy as np
import matplotlib.pyplot as plt
import time

In [6]:
rootdir = '/projects/SOCCOM/datasets/LargeMarineEcos/LME66/'
filename = 'LMEs66.shp'
df = gpd.read_file(rootdir+filename)
grid = xr.open_dataset(ppeDict['rootdir']+ppeDict['gridfile'])
# Reorganize geolon coordinate
grid['geolon'] = grid['geolon'].where(grid['geolon']>-180,grid['geolon']+360)

In [7]:
print(df.head())

   OBJECTID  LME_NUMBER                 LME_NAME GROUPING  ARCTIC USLMES  \
0         1        23.0               Baltic Sea      NaN     NaN    NaN   
1         2        52.0           Sea of Okhotsk      NaN     NaN    NaN   
2         3        22.0                North Sea      NaN     NaN    NaN   
3         4         2.0           Gulf of Alaska      NaN     NaN    Yes   
4         5         9.0  Labrador - Newfoundland   Arctic  Arctic    NaN   

   Shape_Leng  Shape_Area    SUM_GIS_KM  \
0  219.326402   61.950021  3.951472e+05   
1  145.932989  213.796584  1.561297e+06   
2  209.486403  103.965247  6.938472e+05   
3  495.473780  211.045622  1.530668e+06   
4  270.847479  114.597691  9.077923e+05   

                                            geometry  
0  POLYGON ((10.97944 54.38055, 10.98611 54.37805...  
1  POLYGON ((156.70705 51.18858, 156.62592 51.160...  
2  POLYGON ((5.27889 61.98027, 5.24917 61.97388, ...  
3  POLYGON ((-149.41141 61.49638, -149.40311 61.4...  
4  MULTIP

In [10]:
ds = xr.Dataset(coords=grid['geolon'].coords)
gridx,gridy = grid['geolon'].values,grid['geolat'].values
xy_arr = np.vstack((gridx.ravel(), gridy.ravel())).T
# Set up numpy dictionary
inside_dict = {}
for index,row in df.iterrows():
    name = row['LME_NAME']
    inside_dict[name]=np.full(shape = len(xy_arr), fill_value = False)
# Populate np arrays with boolean based on whether point in polygon
for i,xy in enumerate(xy_arr):
    if i%100==0:
        print(str(i)+' of '+str(len(xy_arr)))
    for name,inside in inside_dict.items():
        poly = df[df['LME_NAME']==name]['geometry'].iloc[0]
        inside[i]=poly.contains(Point(xy))

0 of 414720
0.10583114624023438
0.0977025032043457
0.0959017276763916
0.09860658645629883
0.09664487838745117
0.09583187103271484
0.09499788284301758
0.09607410430908203
0.09536147117614746
0.09813427925109863
0.09282660484313965
0.09331130981445312
0.09184432029724121
0.09853434562683105
0.09435653686523438
0.09398436546325684
0.09227299690246582
0.0927119255065918
0.09354114532470703
0.09100031852722168
0.09283971786499023
0.09265589714050293
0.09148573875427246
0.0913243293762207
0.09164190292358398
0.09345626831054688
0.09250640869140625
0.09157395362854004
0.09065103530883789
0.09280586242675781
0.09279346466064453
0.09747028350830078
0.09802889823913574
0.09642314910888672
0.09426188468933105
0.09202361106872559
0.09351634979248047
0.0915975570678711
0.09130740165710449
0.09062838554382324
0.09248113632202148
0.09223008155822754
0.09130382537841797
0.09097027778625488
0.0926368236541748
0.09195280075073242
0.09152984619140625
0.09494400024414062
0.09453344345092773
0.092398166656

KeyboardInterrupt: 

In [ ]:
# Reshape boolean and store in Dataset
for name,inside in inside_dict.items():
    da=xr.DataArray(inside.reshape(gridx.shape).T,dims=ds.dims,coords=ds.coords)
    ds[name]=da

In [ ]:
ds.to_netcdf(ppeDict['datasavedir']+'raw/LME66.ESM4.nc')

In [4]:
time.time()

NameError: name 'time' is not defined